
**Geração de Resumo de notícias - Extração**


1.   Definir fonte das notícias (NewsAPI);
2.   Restringir área de atuação (Brazil - daily headlines ;
3.   Definir os campos de interesse (Título=_title_/Conteúdo completo=_content_);


In [ ]:
#!pip install newsapi-python - instala cliente do site

from newsapi import NewsApiClient
import pandas as pd


#init atribui a pai key cedida pelo site a uma variável
newsapi = NewsApiClient(api_key='a1b8895308db402fa4f3bdc5ba602871')

#sources = newsapi.get_sources() gera uma variável com todas as fontes disponiveis (é bastabte coisa)

#busca todos os artigos da fonte "O Globo", cujo assunto seja "Science"
all_articles = newsapi.get_everything(sources='globo', q='science')

#print(all_articles) para localizar os rotulos das informações necessárias

# Converta os dados em um DataFrame, Salve em um  CSV para ir acompanhando
df = pd.DataFrame(all_articles['articles'])
df.to_csv('news_data.csv', index=False)




**Geração de resumo de notícias - Transformação**

1.  Pré-processamento do texto:
    
    * Limpeza do texto (remover caracteres especiais, pontuações e formatações HTML que possam atrapalhar a IA;
    * Tokenização (separar o texto em palavras ou frases significativas, os tokens);
    * Remoção de Stopwords (remoão de palavrascomuns que não contribuem para o significado do texto)

2. Geração de resumos

    * Caçar uma ia que faça isso pra gente no hugging face;
    * testada duas e a que obteve melhor resultados foi a _facebook/bart-large-cnn_ em detrimnenbto a _phpaiola/ptt5-base-summ-xlsum_

3. Avaliação e aperfeiçoamento

    * Resumos analisados pela mecânica de token do código ficaram bons, porém os títulos não, o ideal é manter os títulos originais.




In [ ]:
#!pip install unidecode
#!pip install nltk
#!pip install transformers

import re
import nltk
from nltk.tokenize import word_tokenize
from unidecode import unidecode
from transformers import BartTokenizer, BartForConditionalGeneration

# Inicializar o tokenizer e o modelo para sumarização
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Função para remoção de caracteres especiais
def remove_special_characters(text):
    text = unidecode(re.sub(r'[^\w\s]', ' ', text))
    text = ' '.join(text.split())
    return text

# Função para tokenização do texto
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

def generate_summary(text):
    inputs = tokenizer.encode(text, max_length=512, truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs, max_length=150, num_beams=5, no_repeat_ngram_size=3, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Aplicar a função de remoção de caracteres especiais às colunas 'title' e 'content'
#df['title'] = df['title'].apply(remove_special_characters)
#df['content'] = df['content'].apply(remove_special_characters)

# Aplicar a função de tokenização às colunas 'title' e 'content'
#df['title_tokens'] = df['title'].apply(tokenize_text)
#df['content_tokens'] = df['content'].apply(tokenize_text)

# Aplicar a função de sumarização às colunas 'title_tokens' e 'content_tokens'
df['title_summary'] = df['title_tokens'].apply(lambda x: generate_summary(' '.join(x)))
df['content_summary'] = df['content_tokens'].apply(lambda x: generate_summary(' '.join(x)))

# Salvar o DataFrame de volta em um novo CSV
df.to_csv('dados_sumarizados.csv', index=False)



**Carregamento / Load **

1. Destino dos resumos
    * Para fins de aprendizado, vou gerar um CSV com o Título Original e a Matéria resumida, juntamente com o Link da noticia. Nesse aspecto, tenho de recurar algumas informações do inicio do trabalho;

2. Inserção dos Resumos
    * Se fosse num recurso externo, haveiriam mais preeocupações, mas inicialmente ficara apenas em CSV mesmo

In [46]:
# Carregar os arquivos
df_sumarizado = pd.read_csv('dados_sumarizados 2 ET.csv')
df_original = pd.read_csv('news_data E.csv')

# Selecionar as colunas relevantes
df_sumarizado = df_sumarizado[['title', 'content_summary']]
df_original = df_original[['title', 'url']]

# Realizar a junção dos dataframes
df_final = pd.merge(df_sumarizado, df_original, on='title', how='inner')

# Salvar o novo dataset
df_final.to_csv('novo_dataset ETL.csv', index=False)